In [1]:
import pandas as pd
import numpy as np
import pywt
import logging
import os
import torch
from torch import nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, WeightedRandomSampler,Dataset
from torchinfo import summary
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn import decomposition



In [2]:
def calibrate_single_phase(phases):
    """
    Calibrate phase data from the single time moment
    Based on:
        https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/sys031fp.pdf
        https://github.com/ermongroup/Wifi_Activity_Recognition/.../phase_calibration.m

    :param phases: phase in the single time moment, np.array of shape(1, num of subcarriers)
    :return: calibrate phase, np.array of shape(1, num of subcarriers)
    """

    phases = np.array(phases)
    difference = 0

    calibrated_phase, calibrated_phase_final = np.zeros_like(phases), np.zeros_like(phases)
    calibrated_phase[0] = phases[0]

    phases_len = phases.shape[0]

    for i in range(1, phases_len):
        temp = phases[i] - phases[i - 1]

        if abs(temp) > np.pi:
            difference = difference + 1 * np.sign(temp)

        calibrated_phase[i] = phases[i] - difference * 2 * np.pi

    k = (calibrated_phase[-1] - calibrated_phase[0]) / (phases_len - 1)
    b = np.mean(calibrated_phase)

    for i in range(phases_len):
        calibrated_phase_final[i] = calibrated_phase[i] - k * i - b

    return calibrated_phase_final

def calibrate_phase(phases):
    """
    Calibrate phase data based on the following method:
        https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/sys031fp.pdf
        https://github.com/ermongroup/Wifi_Activity_Recognition/.../phase_calibration.m

    :param phases: np.array of shape(data len, num of subcarries)
    :return: calibrated phases: np.array of shape(data len, num of subcarriers)
    """

    calibrated_phases = np.zeros_like(phases)

    for i in range(phases.shape[0]):
        calibrated_phases[i] = calibrate_single_phase(np.unwrap(phases[i]))

    return calibrated_phases

def calibrate_amplitude(amplitudes, rssi=1):
    """
    Simple amplitude normalization, that could be multiplied by rsii
    ((data - min(data)) / (max(data) - min(data))) * rssi

    :param amplitudes: np.array of shape(data len, num of subcarriers)
    :param rssi: number
    :return: normalized_amplitude: np.array of shape(data len, num of subcarriers)
    """

    amplitudes = np.array(amplitudes)
    return ((amplitudes - np.min(amplitudes)) / (np.max(amplitudes) - np.min(amplitudes))) * rssi


def calibrate_amplitude_custom(amplitudes, min_val, max_val, rssi=1):
    amplitudes = np.array(amplitudes)
    return ((amplitudes - min_val) / (max_val - min_val)) * rssi



def dwn_noise(vals):
    data = vals.copy()
    threshold = 0.06  # Threshold for filtering

    w = pywt.Wavelet('sym5')
    maxlev = pywt.dwt_max_level(data.shape[0], w.dec_len)

    coeffs = pywt.wavedec(data, 'sym5', level=maxlev)

    for i in range(1, len(coeffs)):
        coeffs[i] = pywt.threshold(coeffs[i], threshold * max(coeffs[i]))

    datarec = pywt.waverec(coeffs, 'sym5')

    return datarec


def hampel(vals_orig, k=7, t0=3):
    # Make copy so original not edited
    vals = pd.Series(vals_orig.copy())

    # Hampel Filter
    L = 1.4826

    rolling_median = vals.rolling(k).median()
    difference = np.abs(rolling_median - vals)
    median_abs_deviation = difference.rolling(k).median()
    threshold = t0 * L * median_abs_deviation
    outlier_idx = difference > threshold
    vals[outlier_idx] = rolling_median

    # print("vals: ", vals.shape)
    return vals.to_numpy()



In [3]:
class CNN_BiLSTM_Attention(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, dropout_rate, bidirectional, output_dim, seq_dim):
        super(CNN_BiLSTM_Attention, self).__init__()

        # CNN layers with ReLU activation, BatchNorm, and MaxPool
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3, 3), padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.maxpool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)
        
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.maxpool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)

        # Compute H and W dynamically
        self.H = input_dim // 4  # Approximate feature map size after pooling
        self.W = seq_dim // 4

        # BiLSTM
        lstm_input_size = 64 * self.H
        self.bilstm = nn.LSTM(
            input_size=lstm_input_size,
            hidden_size=hidden_dim,
            num_layers=layer_dim,
            dropout=dropout_rate if layer_dim > 1 else 0,
            bidirectional=bidirectional,
            batch_first=True
        )

        # Attention mechanism
        self.attention = nn.Sequential(
            nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, 128),
            nn.Tanh(),
            nn.Linear(128, 1)
        )

        # Fully connected layer
        self.fc_out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_rate)  # Extra dropout for regularization

    def forward(self, x):
        # CNN Processing
        x = torch.relu(self.bn1(self.conv1(x)))
        x = self.maxpool1(x)  # Apply MaxPool after first conv block
        
        x = torch.relu(self.bn2(self.conv2(x)))
        x = self.maxpool2(x)  # Apply MaxPool after second conv block

        # Reshape for BiLSTM
        batch_size = x.size(0)
        x = x.permute(0, 3, 1, 2).contiguous().view(batch_size, self.W, -1)

        # BiLSTM Processing
        lstm_out, _ = self.bilstm(x)  # Shape: (batch, W, hidden_dim * num_directions)

        # Attention Weights
        attn_scores = self.attention(lstm_out)  # Shape: (batch, W, 1)
        attn_weights = torch.softmax(attn_scores.squeeze(-1), dim=-1)  # Shape: (batch, W)

        # Context Vector
        context_vector = torch.sum(lstm_out * attn_weights.unsqueeze(-1), dim=1)

        # Fully Connected Layer
        out = self.fc_out(self.dropout(context_vector))
        return out


In [4]:
SUBCARRIES_NUM=52

def read_csi_data_from_csv(path_to_csv,  antenna_pairs=1):
    """
    Read csi data(amplitude, phase) from .csv data

    :param path_to_csv: string
    :param is_five_hhz: boolean
    :param antenna_pairs: integer
    :return: (amplitudes, phases) => (np.array of shape(data len, num_of_subcarriers * antenna_pairs),
                                     np.array of shape(data len, num_of_subcarriers * antenna_pairs))
    """

    data = pd.read_csv(path_to_csv, header=None).values
    # print("shape",data.shape)

    
    subcarries_num = SUBCARRIES_NUM

    # 1 -> to skip subcarriers numbers in data
    amplitudes = data[:, :subcarries_num *  antenna_pairs]
    phases = data[:, subcarries_num * antenna_pairs:]

    return amplitudes, phases


def read_labels_from_csv(path_to_csv, expected_length):
    """
    Read labels (human activities) from csv file and remove unwanted classes.
    
    :param path_to_csv: string
    :return: filtered labels, np.array of shape (data_len, 1)
    """
    data = pd.read_csv(path_to_csv, header=None).values
    labels = data[:, 1]
    
    # Ensure labels match the expected length
    if len(labels) > expected_length:
        labels = labels[:expected_length]  # Trim extra labels if any
    elif len(labels) < expected_length:
        raise ValueError(f"Label file {path_to_csv} has fewer rows than CSI data!")

    return labels



In [5]:
SUBCARRIES_NUM=52


PHASE_MIN, PHASE_MAX = 3.1389, 3.1415
AMP_MIN, AMP_MAX = 0.0, 577.6582


def csi_data(csis):
    """
    Read csi data(amplitude, phase) from .csv data

    :param path_to_csv: string
    :param is_five_hhz: boolean
    :param antenna_pairs: integer
    :return: (amplitudes, phases) => (np.array of shape(data len, num_of_subcarriers * antenna_pairs),
                                     np.array of shape(data len, num_of_subcarriers * antenna_pairs))
    """
    subcarries_num = SUBCARRIES_NUM
    amplitudes_list = []
    phases_list = []

    for data in csis:
        if len(data) != subcarries_num  * 2:
            raise ValueError(f"Data length mismatch: expected {subcarries_num * 2}, got {len(data)}")

        amplitudes = data[:subcarries_num ]
        phases = data[subcarries_num :]
        
        amplitudes_list.append(amplitudes)
        phases_list.append(phases)

    return np.array(amplitudes_list), np.array(phases_list)
        



class CSIDataset(Dataset):
    """CSI Dataset."""

    def __init__(self, train_csi, labels, window_size=32, step=1,is_training=False):
        self.is_training = is_training
        self.amplitudes, self.phases= csi_data(train_csi)
        # print("len",len(self.amplitudes))
        # print(self.phases.shape)
        self.labels = labels
 
        self.amplitudes = calibrate_amplitude(self.amplitudes)

        pca = decomposition.PCA(n_components=12)

        self.amplitudes_pca = []

        data_len = self.phases.shape[0]
        print('data_len',data_len)
        for i in range(self.phases.shape[1]):
            self.amplitudes[:data_len, i] = dwn_noise(hampel(self.amplitudes[:, i]))[
                :data_len
            ]
        # print("Amplitudes shape:", self.amplitudes.shape)
        self.amplitudes_pca = pca.fit_transform(self.amplitudes)
        # print("PCA output shape:", self.amplitudes_pca.shape)
        self.explained_variance_ratio = pca.explained_variance_ratio_
        print("Explained variance ratio:", self.explained_variance_ratio)
        self.cumulative_explained_variance_ratio = np.cumsum(
            self.explained_variance_ratio
        )
        print("Cumulative explained variance ratio:", self.cumulative_explained_variance_ratio)
        self.amplitudes_pca = np.array(self.amplitudes_pca)
        self.label_keys = list(set(self.labels))
        self.class_to_idx = {
            "standing": 0,
            "walking": 1,
            "jumping": 2,
            "no_person": 3,
        }
        self.idx_to_class = {v: k for k, v in self.class_to_idx.items()}

        self.window = window_size
        if window_size == -1:
            self.window = self.labels.shape[0] - 1

        self.step = step

    def __getitem__(self, idx):
        if self.window == 0:
            return (
                np.append(self.amplitudes[idx], self.phases[idx]),
                self.class_to_idx[self.labels[idx + self.window - 1]],
            )

        idx = idx * self.step
        all_xs = []

        for index in range(idx, idx + self.window):
        # Load the amplitude and PCA data for this timestep
            amplitude = self.amplitudes[index]
            phase= self.phases[index]
            pca = self.amplitudes_pca[index]

            # Combine features
            combined = np.append(phase, amplitude)

            # Add noise to this individual sample 
            if self.is_training:
                noise = np.random.normal(0, 0.01, size=combined.shape)
                combined += noise

            all_xs.append(combined)


        return np.array(all_xs), self.class_to_idx[self.labels[idx + self.window - 1]]

    def __len__(self):
        length = max(1, int((self.labels.shape[0] - self.window) // self.step) + 1)
        print(f"CSIDataset length: {length}")  # Debugging
        return length


In [6]:

# Configure logging
log_filename = "training.log"
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler(log_filename),  # Log to a file
        logging.StreamHandler(),  # Log to the console
    ],
)

# Cuda support
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

logging.info("Device: {}".format(device))

# Define dataset structure
# DATASET_FOLDER = "/content/drive/MyDrive/data"
DATASET_FOLDER="E:\\Wifi_based_HAR\\preprocessing\\merged"

# LSTM Model parameters
input_dim = 104  
hidden_dim = 512
layer_dim = 2
output_dim = 4
dropout_rate = 0.2
bidirectional = False
SEQ_DIM = 1024
DATA_STEP = 4

BATCH_SIZE = 64
EPOCHS_NUM = 100
LEARNING_RATE = 0.0001

def save_checkpoint(state, filename="checkpoint.pth"):
    torch.save(state, filename)
    logging.info(f"Checkpoint saved: {filename}")


def load_checkpoint(filename="checkpoint.pth"):
    if torch.cuda.is_available():
        checkpoint = torch.load(filename)
    else:
        checkpoint = torch.load(filename, map_location=torch.device("cpu"))

    logging.info(f"Checkpoint loaded: {filename}")
    return checkpoint




def read_all_data_from_files(data_path, label_path, antenna_pairs=1):
    """
    Read CSI and labels from merged CSV files and apply scaling to amplitudes and phases.
    """
    amplitudes, phases = read_csi_data_from_csv(data_path, antenna_pairs)
    labels = read_labels_from_csv(label_path, len(amplitudes))

    # Convert to numpy arrays if not already
    amplitudes = np.array(amplitudes)
    print("Amplitudes Shape:", amplitudes.shape)
    print("max:", np.max(amplitudes))
    print("min:", np.min(amplitudes))
    phases = np.array(phases)
    print("Phases Shape:", phases.shape)
    print("max:", np.max(phases))
    print("min:", np.min(phases))

    # Initialize scalers
    amp_scaler = MinMaxScaler()
    phase_scaler = MinMaxScaler(feature_range=(-1, 1))  # For phases (-π to π)

    # Reshape if necessary (for 1D arrays)
    if len(amplitudes.shape) == 1:
        amplitudes = amplitudes.reshape(-1, 1)
    if len(phases.shape) == 1:
        phases = phases.reshape(-1, 1)

    # Apply scaling
    scaled_amplitudes = amp_scaler.fit_transform(amplitudes)
    print("Scaled Amplitudes Shape:", scaled_amplitudes.shape)
    print("scaled max:", np.max(scaled_amplitudes))
    print("scaled min:", np.min(scaled_amplitudes))
    scaled_phases = phase_scaler.fit_transform(phases)
    print("Scaled Phases Shape:", scaled_phases.shape)
    print("scaled max:", np.max(scaled_phases))
    print("scaled min:", np.min(scaled_phases))
    return scaled_amplitudes, scaled_phases, labels

def get_class_weights(labels):
    """Compute inverse class frequencies to balance sampling."""
    class_to_idx = {
        "standing": 0,
        "walking": 1,
        "jumping": 2,
        "no_person": 3,
    }

    # Convert string labels to integers
    labels = np.array([class_to_idx[label] for label in labels])
    class_counts = np.bincount(labels)  # Count occurrences of each class
    total_samples = len(labels)
    class_weights = total_samples / (len(class_counts) * class_counts)  # Inverse frequency
    sample_weights = np.array([class_weights[label] for label in labels])
    return sample_weights

def load_data():
    # Load merged CSI data and labels
    data_path = os.path.join(DATASET_FOLDER, "data.csv")
    label_path = os.path.join(DATASET_FOLDER, "label.csv")
    amplitudes, phases, labels = read_all_data_from_files(data_path, label_path)

    # print("Label shape:", labels.shape)
    # print("Amplitudes shape:", amplitudes.shape)

    # Concatenate amplitude and phase data for input features
    csi_data = np.hstack((amplitudes, phases))

    # Shuffle the dataset before splitting (optional, useful if classes are sequential)
    indices = np.arange(len(labels))
    np.random.shuffle(indices)
    csi_data, labels = csi_data[indices], labels[indices]

    # Stratified split to maintain class balance
    train_csi, val_csi, train_labels, val_labels = train_test_split(
        csi_data, labels, test_size=0.2, stratify=labels, random_state=42
    )
    
    unique, counts = np.unique(train_labels, return_counts=True)
    print("Train class distribution:", dict(zip(unique, counts)))

    unique, counts = np.unique(val_labels, return_counts=True)
    print("Validation class distribution:", dict(zip(unique, counts)))


    # Compute normalization stats on training set
    train_mean = np.mean(train_csi, axis=0)
    train_std = np.std(train_csi, axis=0) + 1e-8  # Avoid division by zero
    train_csi = (train_csi - train_mean) / train_std
    val_csi = (val_csi - train_mean) / train_std

    # Create datasets
    train_dataset = CSIDataset(
        train_csi, train_labels, window_size=SEQ_DIM, step=DATA_STEP, is_training=True
    )
    val_dataset = CSIDataset(
        val_csi, val_labels, window_size=SEQ_DIM, step=DATA_STEP, is_training=False
    )
    print(f"Dataset size: {len(train_dataset)}")
    print(f"Max index in dataset: {max(range(len(train_dataset)))}")
    print(f"Max index in dataset: {max(range(len(val_dataset)))}")

    
    # Compute weights for stratified sampling
    # Adjust sample weights to match the new dataset size
    windowed_labels = [train_labels[idx + SEQ_DIM - 1] for idx in range(0, len(train_labels) - SEQ_DIM, DATA_STEP)]
    sample_weights = get_class_weights(np.array(windowed_labels))
    
    # num_samples = len(train_dataset)
    
    sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)
    print(f"Sample Weights Shape: {sample_weights.shape}")
    print(f"Unique Weights: {np.unique(sample_weights)}")
    print(f"Sampler Length: {len(sampler)}")

    # DataLoaders
    trn_dl = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=sampler, drop_last=True)
    val_dl = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

    return trn_dl, val_dl


def train():
    save_dir = "saved_models"
    os.makedirs(save_dir, exist_ok=True)

    # Initialize metrics
    patience, trials, best_acc = 10, 0, 0
    trn_dl, val_dl = load_data()

    # Ensure input dimensions are valid
    assert SEQ_DIM % 4 == 0, "SEQ_DIM must be divisible by 4 for CNN operations"
    assert input_dim % 4 == 0, "input_dim must be divisible by 4 for CNN operations"

    # Model initialization
    model = CNN_BiLSTM_Attention(
        input_dim=input_dim,
        hidden_dim=hidden_dim,
        layer_dim=layer_dim,
        dropout_rate=dropout_rate,
        bidirectional=bidirectional,
        output_dim=output_dim,
        seq_dim=SEQ_DIM
    ).to(device)
    
    # Print model summary
    summary(model, input_size=(1, 1, input_dim, SEQ_DIM), device=device)

    # Loss, optimizer, and scheduler
    # class_weights = torch.tensor(class_weights).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
    scheduler = ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=3, verbose=True)

    checkpoint_filename = os.path.join(save_dir, "checkpoint.pth")
    start_epoch = 1

    # Load checkpoint if available
    if os.path.exists(checkpoint_filename):
        try:
            checkpoint = torch.load(checkpoint_filename, map_location=device)
            model.load_state_dict(checkpoint["model_state_dict"])
            optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
            scheduler.load_state_dict(checkpoint["scheduler_state_dict"])
            start_epoch = checkpoint["epoch"] + 1
            best_acc = checkpoint["best_acc"]
            logging.info(f"Resumed training from epoch {start_epoch}")
        except Exception as e:
            logging.error(f"Failed to load checkpoint: {str(e)}. Starting fresh training.")

    # Training loop
    for epoch in range(start_epoch, EPOCHS_NUM + 1):
        model.train()
        epoch_loss, correct, total = 0, 0, 0

        for i, (x_batch, y_batch) in tqdm(enumerate(trn_dl), total=len(trn_dl), desc=f"Epoch {epoch}"):
            x_batch = x_batch.unsqueeze(1).float().to(device)  # Ensure correct shape
            y_batch = y_batch.long().to(device)

            optimizer.zero_grad()
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)

            # Backpropagation
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Apply gradient clipping
            optimizer.step()

            # Metrics tracking
            epoch_loss += loss.item() * x_batch.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == y_batch).sum().item()
            total += y_batch.size(0)

        # Compute training metrics
        train_loss = epoch_loss / total
        train_acc = correct / total

        # Validation phase
        model.eval()
        val_loss, correct, total = 0, 0, 0
        with torch.no_grad():
            for x_val, y_val in val_dl:
                x_val = x_val.unsqueeze(1).float().to(device)
                y_val = y_val.long().to(device)

                outputs = model(x_val)
                loss = criterion(outputs, y_val)

                val_loss += loss.item() * x_val.size(0)
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == y_val).sum().item()
                total += y_val.size(0)

        val_loss /= len(val_dl.dataset)
        val_acc = correct / total

        # Learning rate scheduler
        scheduler.step(val_loss)

        logging.info(
            f"Epoch {epoch:3d} | Validation Loss: {val_loss:.4f}, "
            f"Validation Acc.: {val_acc:.2%}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2%}"
        )
        print(f"Epoch {epoch:3d} | Validation Loss: {val_loss:.4f}, "
            f"Validation Acc.: {val_acc:.2%}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2%}")
        if val_acc > best_acc:
            trials = 0
            best_acc = val_acc
            torch.save(model.state_dict(), os.path.join(save_dir, "lstm_classifier_best.pth"))
            logging.info(f"Epoch {epoch}: Best model saved with accuracy {best_acc:.2%}")
        else:
            trials += 1
            if trials >= patience:
                logging.info(f"Early stopping at epoch {epoch}")
                break

        # Save checkpoint
        torch.save({
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "scheduler_state_dict": scheduler.state_dict(),
            "best_acc": best_acc,
        }, checkpoint_filename)

        scheduler.step(val_loss)
        
        # Logging
        logging.info
        
if __name__ == "__main__":
    train()


2025-02-28 13:21:16,434 - INFO - Device: cpu


Amplitudes Shape: (355860, 52)
max: 132.3782459469833
min: 0.0
Phases Shape: (355860, 52)
max: 3.141592653589793
min: -3.1266683885997084
Scaled Amplitudes Shape: (355860, 52)
scaled max: 1.0
scaled min: 0.0
Scaled Phases Shape: (355860, 52)
scaled max: 1.0000000000000002
scaled min: -1.0
Train class distribution: {'jumping': 71172, 'no_person': 71172, 'standing': 71172, 'walking': 71172}
Validation class distribution: {'jumping': 17793, 'no_person': 17793, 'standing': 17793, 'walking': 17793}
data_len 284688
Explained variance ratio: [0.77007663 0.0748044  0.01985271 0.01184649 0.00771351 0.00626069
 0.00447664 0.00425141 0.00399074 0.00356864 0.00351989 0.00333974]
Cumulative explained variance ratio: [0.77007663 0.84488103 0.86473374 0.87658024 0.88429375 0.89055444
 0.89503108 0.89928248 0.90327322 0.90684186 0.91036175 0.91370149]
data_len 71172
Explained variance ratio: [0.77591428 0.07291508 0.01918219 0.0116129  0.0076221  0.00619472
 0.00450321 0.00417915 0.0038408  0.00345344

e:\Wifi_based_HAR\venv\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "
Epoch 1:   0%|          | 2/1108 [01:14<11:27:27, 37.29s/it]


KeyboardInterrupt: 